### First Login
After running this click Sign-In and login to Spotify. If the Time expires then repeat the process again.

In [1]:
from ipyauth import ParamsSpotify, Auth

p = ParamsSpotify(redirect_uri='http://localhost:8888/callback', client_id="9e4657eefbac41afa98c61f590d8fd51")
auth = Auth(params=p)
auth

Auth(children=(HBox(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x04\x00\x00\x00\x02\xe…

In [2]:
access_token = auth.access_token
access_token

'BQAMPeX3nsnfMEjVOPO479L12VYMgMPn75xuwaGFQSeIVsFkQBIYzSHC-RXQZU_gLOgsBWx4YLJ20mpWoA5HTrtBsTLVDMn93QSSpmUBOCXwAACV4IFtRh_uVagap1DTe414oyb_v5ZySI9lRz3AghA6cKM35HPaWrIeGfzknXatvfy8-vco_N0VS7d7Xl32HYaNF61HwwHdFTYsHA6tBVZ2V5xcoK6d1jYmbKC8ea9Jm6T0Lj-VtSTVXYw'

### Common Stuff and Imports


In [5]:
!pip install  pandas

     |████████████████████████████████| 10.4MB 1.8MB/s eta 0:00:01    |███████████████████████         | 7.5MB 1.5MB/s eta 0:00:02
     |████████████████████████████████| 512kB 712kB/s eta 0:00:01
     |████████████████████████████████| 20.0MB 378kB/s eta 0:00:01     |█████████████████████           | 13.1MB 1.4MB/s eta 0:00:06     |███████████████████████         | 14.4MB 1.1MB/s eta 0:00:06


In [6]:
import requests
from IPython.display import Image
from pandas.io.json import json_normalize
from pandas import DataFrame,read_pickle
import logging
import os

logger = logging.getLogger()
logger.setLevel(logging.INFO)

def fetch( path, url=None ):
    callPath = url if (url!=None) else ("https://api.spotify.com" + path)
    response = requests.get(callPath , headers= {"Authorization":"Bearer " + access_token })
    if (response.status_code!=200):
        logging.debug("error ")
    return response.json()

def fetchPage( path, offset, limit):
    res = fetch(  path + "?offset=" + str(offset) + "&limit=" + str(limit))
    return res

def fetchAll( path ):
    more = fetchPage( path, 0, 50)
    limit = more["limit"]
    total = more["total"] - limit
    items = more["items"]
    while((total>0) and (more["next"]!=None )):
        more = fetch( None, url = more["next"] )
        items.extend( more["items"])
        total = total - len(more["items"])
    return items

def fetchPageIds( path, ids): 
  res = fetch("{0}?ids={1}".format(path,",".join(ids)))
  return res

def fetchAllIds( path, resultField, ids, pageSize=50, existingDf=None):
    if (existingDf is not None):
        keys = existingDf.index.values
        ids = list(filter( lambda id: (id not in keys),ids))
        existingDf = existingDf.reset_index()
        logging.info("Filtering out {0} IDs. Now {1}".format(len(keys), len(ids)))
    total = len(ids)
    logging.info("Requesting {0} rows. {1} ... {2}".format(total, path, resultField))
    offset = 0
    while (offset < total) :
        result = fetchPageIds(path, ids[offset: min(total, offset + pageSize)])
        items = json_normalize(result[resultField])
        if (existingDf is None):
            logging.info("Creating new DF {0}".format(len(items)))
            existingDf = items
        else:
            existingDf = existingDf.append(items, ignore_index=True )
        offset += len(items)
        
    return existingDf.to_dict(orient="records")

user = fetch("/v1/me")
Image(url=user["images"][0]["url"], width=100)

### User Retreival
Get the user data and show the image

In [6]:
data = fetchAll("/v1/me/tracks")
tracksDf = json_normalize(data, sep="_").set_index("track_id")
tracksDf.to_pickle("mytracks.pkl")
tracksDf.head()

,added_at,track_album_album_type,track_album_artists,track_album_available_markets,track_album_external_urls_spotify,track_album_href,track_album_id,track_album_images,track_album_name,track_album_release_date,...,track_popularity,track_preview_url,track_track_number,track_type,track_uri,track_linked_from_external_urls_spotify,track_linked_from_href,track_linked_from_id,track_linked_from_type,track_linked_from_uri
track_id,,,,,,,,,,,,,,,,,,,,,
1CS7Sd1u5tWkstBhpssyjP,2019-10-26T01:46:13Z,album,[{'external_urls': {'spotify': 'https://open.s...,"[AU, CA, US]",https://open.spotify.com/album/4Pv7m8D82A1Xun7...,https://api.spotify.com/v1/albums/4Pv7m8D82A1X...,4Pv7m8D82A1Xun7xNCKZjJ,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Hozier (Expanded Edition),2014-09-19,...,78,https://p.scdn.co/mp3-preview/cdbb37ee18245800...,1,track,spotify:track:1CS7Sd1u5tWkstBhpssyjP,NaN,NaN,NaN,NaN,NaN
38Qqq67n2Z6yGANpKlCuKv,2019-10-25T17:19:41Z,album,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",https://open.spotify.com/album/2ESmheV4tLGuJil...,https://api.spotify.com/v1/albums/2ESmheV4tLGu...,2ESmheV4tLGuJilEJYk02Q,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Philip Glass: The Fall of the House of Usher,2019-10-18,...,25,https://p.scdn.co/mp3-preview/57b67f5ba41e2f61...,6,track,spotify:track:38Qqq67n2Z6yGANpKlCuKv,NaN,NaN,NaN,NaN,NaN
3RMuJ4qdkgs71mBNbPpZu0,2019-10-25T16:25:21Z,single,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",https://open.spotify.com/album/691WK2nkTQrNlq9...,https://api.spotify.com/v1/albums/691WK2nkTQrN...,691WK2nkTQrNlq9egbsJQw,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Swimmers,2019-10-25,...,48,https://p.scdn.co/mp3-preview/24e431807bfd082a...,1,track,spotify:track:3RMuJ4qdkgs71mBNbPpZu0,NaN,NaN,NaN,NaN,NaN
2WbrVeftLDcSAfQBHxA9P4,2019-10-25T16:15:45Z,single,[{'external_urls': {'spotify': 'https://open.s...,"[CA, US]",https://open.spotify.com/album/1sWcGlojlFQctnf...,https://api.spotify.com/v1/albums/1sWcGlojlFQc...,1sWcGlojlFQctnfRWlmUjI,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Blind Leading The Blind,2019-10-23,...,63,https://p.scdn.co/mp3-preview/e27317bf8a908a44...,1,track,spotify:track:2WbrVeftLDcSAfQBHxA9P4,NaN,NaN,NaN,NaN,NaN
48reQYlu36EpB24R9rIlxT,2019-10-25T16:08:23Z,single,[{'external_urls': {'spotify': 'https://open.s...,"[AE, AR, AU, BH, BO, BR, CA, CL, CO, CR, DO, E...",https://open.spotify.com/album/5HaKhstW9fj5wBB...,https://api.spotify.com/v1/albums/5HaKhstW9fj5...,5HaKhstW9fj5wBBkJEJx6t,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",One Slip (Remix 2019),2019-10-25,...,52,https://p.scdn.co/mp3-preview/a4624453758f6517...,1,track,spotify:track:48reQYlu36EpB24R9rIlxT,NaN,NaN,NaN,NaN,NaN


In [7]:
tracksDf = read_pickle("mytracks.pkl")
travk = tracksDf[["added_at","track_name","track_album_name"]]
#travk.to_dict(orient="records")
travk.head()

,added_at,track_name,track_album_name
track_id,,,
1CS7Sd1u5tWkstBhpssyjP,2019-10-26T01:46:13Z,Take Me to Church,Hozier (Expanded Edition)
38Qqq67n2Z6yGANpKlCuKv,2019-10-25T17:19:41Z,Act I Interlude,Philip Glass: The Fall of the House of Usher
3RMuJ4qdkgs71mBNbPpZu0,2019-10-25T16:25:21Z,Swimmers,Swimmers
2WbrVeftLDcSAfQBHxA9P4,2019-10-25T16:15:45Z,Blind Leading The Blind,Blind Leading The Blind
48reQYlu36EpB24R9rIlxT,2019-10-25T16:08:23Z,One Slip - Remix 2019,One Slip (Remix 2019)


### Create a Track to Artist Table
Pick out the track.artists array for each library track record. The meta (parent record) is the track.id. Use a prefix for both the meta and the record because they both use id.


In [39]:
#data[0]
artist_and_track = json_normalize( data=data, record_path=['track','artists'],  meta=[["track","id"]],  record_prefix='artist_',   sep="_" )
artist_and_track = artist_and_track[['track_id','artist_id']]
data[0].keys()

#len(artist_and_track["artist_id"].values)
#len(list(set(list(artist_and_track["sp_artist_id"].values))))

dict_keys(['added_at', 'track'])

### Read the Artists
Load the artists one by one. Use the Pickle File **artists.pkl** as a cache.
* Create a DataFrame using the cache file if one exists (otherwise None)
* Get the list of all unique artist_ids from the previous DF
* Call `fetchAllIds` - using the artists path, the `artists` JSON field path and the cache DF
* Recreate the new artistDf (Dict returned)
* Save the file back to **artists.pkl**

In [51]:
artistsPickle = read_pickle("artists.pkl") if (os.path.isfile("artists.pkl")) else None 
artistIds = list(set(artist_and_track["artist_id"].values))
artists = fetchAllIds("/v1/artists","artists",artistIds,existingDf=artistsPickle)
artistsDf = json_normalize(artists).set_index("id")
artistsDf.to_pickle("artists.pkl")
artistsDf.head()

True

### Read the Albums

In [92]:
albumsPickle = read_pickle("albums.pkl") if (os.path.isfile("albums.pkl")) else None 
album_ids  = list(set(json_normalize( data=data, sep="_" )["track_album_id"].values))
albums = fetchAllIds("/v1/albums","albums",album_ids,pageSize=20,existingDf=albumsPickle)
albumsDf = json_normalize(albums).set_index("id")
albumsDf.to_pickle("albums.pkl")
albumsDf.head()




INFO:root:Filtering out 3284 IDs. Now 0
INFO:root:Requesting 0 rows. /v1/albums ... albums


,album_type,artists,available_markets,copyrights,genres,href,images,label,name,popularity,...,uri,external_ids.upc,external_urls.spotify,tracks.href,tracks.items,tracks.limit,tracks.next,tracks.offset,tracks.previous,tracks.total
id,,,,,,,,,,,,,,,,,,,,,
5MIGplDUloZZdW8izdHzJe,album,[{'external_urls': {'spotify': 'https://open.s...,[],"[{'text': '2009 Strut', 'type': 'C'}, {'text':...",[],https://api.spotify.com/v1/albums/5MIGplDUloZZ...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Strut,New York - Addis - London: The Story of Ethio ...,4,...,spotify:album:5MIGplDUloZZdW8izdHzJe,0730003305122,https://open.spotify.com/album/5MIGplDUloZZdW8...,https://api.spotify.com/v1/albums/5MIGplDUloZZ...,[{'artists': [{'external_urls': {'spotify': 'h...,50,None,0,None,20
0gsKWYzmxZ83Ew4N0XLcz1,album,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...","[{'text': '2017 InFiné', 'type': 'C'}, {'text'...",[],https://api.spotify.com/v1/albums/0gsKWYzmxZ83...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",InFiné,Mirapolis,48,...,spotify:album:0gsKWYzmxZ83Ew4N0XLcz1,3700398717494,https://open.spotify.com/album/0gsKWYzmxZ83Ew4...,https://api.spotify.com/v1/albums/0gsKWYzmxZ83...,[{'artists': [{'external_urls': {'spotify': 'h...,50,None,0,None,12
5qETVGzjqS3dFqzKpDAPgg,album,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...","[{'text': '2017 !K7 Records', 'type': 'C'}, {'...",[],https://api.spotify.com/v1/albums/5qETVGzjqS3d...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",!K7 Records,DJ-Kicks (DJ Tennis) [Mixed Tracks],34,...,spotify:album:5qETVGzjqS3dFqzKpDAPgg,0730003733826,https://open.spotify.com/album/5qETVGzjqS3dFqz...,https://api.spotify.com/v1/albums/5qETVGzjqS3d...,[{'artists': [{'external_urls': {'spotify': 'h...,50,None,0,None,31
3GBnNRYsxBfEeMSMmTpJ25,album,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, BO, BR, CA, CL, CO, CR, DO, DZ, EC, G...","[{'text': '© 2000 Modular Recordings', 'type':...",[],https://api.spotify.com/v1/albums/3GBnNRYsxBfE...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Universal Music Australia Pty. Ltd.,Since I Left You,50,...,spotify:album:3GBnNRYsxBfEeMSMmTpJ25,00898926000422,https://open.spotify.com/album/3GBnNRYsxBfEeMS...,https://api.spotify.com/v1/albums/3GBnNRYsxBfE...,[{'artists': [{'external_urls': {'spotify': 'h...,50,None,0,None,18
4hNpqbuCVDxDxaJJiamGGI,single,[{'external_urls': {'spotify': 'https://open.s...,[US],"[{'text': '© 2009 Virgin Records America, Inc....",[],https://api.spotify.com/v1/albums/4hNpqbuCVDxD...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Virgin Records,Deep Cuts,4,...,spotify:album:4hNpqbuCVDxDxaJJiamGGI,05099969672750,https://open.spotify.com/album/4hNpqbuCVDxDxaJ...,https://api.spotify.com/v1/albums/4hNpqbuCVDxD...,[{'artists': [{'external_urls': {'spotify': 'h...,50,None,0,None,4
